# Hacktiv8 Phase 1: Graded Challenge 2

---

Graded Challenge ini dibuat guna mengevaluasi pembelajaran pada Hacktiv8 Data Science Fulltime Program khususnya pada konsep Classification terutama Logistic Regression dan SVM.

## Introduction

By [Rifky Aliffa](https://github.com/Penzragon)

![weather](https://images.ctfassets.net/hrltx12pl8hq/6TIZLa1AKeBel0yVO7ReIn/1fc0e2fd9fcc6d66b3cc733aa2547e11/weather-images.jpg?fit=fill&w=800&h=300)

### Dataset

Dataset yang digunakan pada project ini adalah dataset yang berisi tentang cuaca harian dari berbagai stasiun cuaca di Australia selama 10 tahun. Dataset ini berisi 145460 baris dengan 23 kolom yang diantaranya adalah date, location, MinTemp, MaxTemp, dan masih banyak lagi. Dataset dapat dilihat di [Kaggle](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package).

### Objectives

**Graded Challenge 2** ini dibuat guna mengevaluasi konsep Logistic Regression dan SVM sebagai berikut:

- Mampu memahami konsep Classification dengan Logistic Regression dan SVM.
- Mampu mempersiapkan data untuk digunakan dalam model Logistic Regression dan SVM.
- Mampu mengimplementasikan Logistic Regression dan SVM untuk membuat prediksi.

## Import Libraries

Pada project kali ini library yang akan digunakan adalah **Pandas**, **Numpy**, **Matplotlib**, **Seaborn**, dan **Scikit-Learn**.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report